In [2]:
import pandas as pd

# Load the dataset
file_path = 'dataset/psc_severity_train.csv'
data = pd.read_csv(file_path)

psc_data_cleaned = data.dropna(subset=['annotation_severity'])
data = psc_data_cleaned[psc_data_cleaned['annotation_severity'] != 'Not a deficiency']

# Compute the most frequent severity for each unique combination of PscInspectionId and deficiency_code
most_frequent = (
    data.groupby(["PscInspectionId", "deficiency_code"])["annotation_severity"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)  # Find the mode (most frequent value)
    .reset_index(name="Most_Frequent_Severity")
)

# Merge with def_text to include the associated text for each combination
most_frequent_with_text = most_frequent.merge(
    data[["PscInspectionId", "deficiency_code", "def_text"]].drop_duplicates(),
    on=["PscInspectionId", "deficiency_code"],
    how="left"
)

# Save or display the final result
most_frequent_with_text.to_csv('most_frequent_severity_with_def_text.csv', index=False)